In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import csv
import requests
import tkinter as tk
from tkinter import scrolledtext
import threading  # <--- Added to prevent the window from freezing

# ---------------- CONSTANTS (Unchanged) ----------------
SAFE = "GREEN"
SUSPICIOUS = "YELLOW"
MALICIOUS = "RED"
chat_history = []  #Stores the last few chat messages so the AI remembers context.

def load_dataset():
    data = {}
    try:
        with open("ip_dataset.csv", "r", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                data[row["ip"]] = row["status"].lower()
    except FileNotFoundError:    #If dataset is missing, silently switch to zero‑day detection mode.
        pass
    return data

#-----------Core IDS/IPS logic.----------------
def ai_decision(ip, dataset):
    if ip in dataset:
        if dataset[ip] == "safe":
            return SAFE, "Recognized trusted IP", "No action required"
        else:
            return MALICIOUS, "Known malicious IP detected", "IP blocked"
    else:
        return SUSPICIOUS, "Unknown IP (Zero-Day)", "IP temporarily blocked and monitored"

# ---------------- UPDATED AI CALL (With Error Catching) ----------------
def get_ai_response(ip, flag, reason, action, question, callback):
    """This function runs in the background so the window doesn't freeze."""

   

    
    global chat_history   #Allows function to modify global chat memory.
    system_prompt = f"""
You are a STRICT IDS/IPS AI assistant.

You ONLY answer questions related to:
- Intrusion Detection Systems (IDS)
- Intrusion Prevention Systems (IPS)
- Network security
- IP threat analysis
- Firewall actions

If the user asks ANY unrelated question
(such as general knowledge, programming, personal, math, jokes, etc.),
you MUST reply exactly: "❌ This question is outside IDS/IPS scope."

IP Address: {ip}
Threat Level: {flag}
Reason: {reason}
Action Taken: {action}
"""
#Injects real scan results into AI context.
    
    chat_history.append(f"User: {question}")
    
    context = "\n".join(chat_history[-6:])   #Keeps only last 6 messages to limit memory size.
    payload = {
        "model": "phi",
        "prompt": f"{system_prompt}\n\n{context}\nAssistant:",
        "stream": False
    }

    try:
        response = requests.post("http://localhost:11434/api/generate", json=payload, timeout=60)
        bot_reply = response.json().get("response", "").strip()
        chat_history.append(f"Assistant: {bot_reply}")
        callback(bot_reply) # Send the answer back to the GUI
    except Exception as e:
        callback(f"Error: Could not connect to Ollama. ({e})")

# ---------------- THE GUI ----------------
class ChatGUI:
    def __init__(self, ip, flag, reason, action):
        self.ip, self.flag, self.reason, self.action = ip, flag, reason, action

        #Main window
        self.root = tk.Tk()
        self.root.title("AI Security Assistant")
        self.root.geometry("600x500")

        # Status Bar
        status_color = "#2ecc71" if flag == "GREEN" else "#f1c40f" if flag == "YELLOW" else "#e74c3c"
        header = tk.Label(self.root, text=f"IP: {ip} | STATUS: {flag}", bg=status_color, fg="white", font=("Arial", 12, "bold"))
        header.pack(fill=tk.X)

        # Chat Display
        self.display = scrolledtext.ScrolledText(self.root, state='disabled', wrap='word', font=("Arial", 10))
        self.display.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        # Input Field
        self.entry = tk.Entry(self.root, font=("Arial", 11))
        self.entry.pack(padx=10, pady=10, fill=tk.X)
        self.entry.bind("<Return>", self.handle_send)
        self.entry.focus_set() # Force the cursor to be in the box

        self.update_display("System", "Ready. Type your question and press Enter.")

    def handle_send(self, event=None):
        user_input = self.entry.get()
        if not user_input: return
        
        self.update_display("You", user_input)
        self.entry.delete(0, tk.END)
        self.update_display("AI", "Thinking...") # Visual feedback

        # Start AI in a BACKGROUND thread so the window stays active
        thread = threading.Thread(target=get_ai_response, args=(
            self.ip, self.flag, self.reason, self.action, user_input, self.on_ai_response
        ))
        thread.daemon = True # Closes thread if you close the window
        thread.start()

        
# ---------------- VALIDATE IDS/IPS QUESTIONS ----------------
    def is_ids_question(question):
        keywords = [
            "ip", "ids", "ips", "attack", "threat", "malicious",
            "firewall", "block", "network", "traffic", "security",
            "port", "scan", "ddos", "intrusion"
        ]
        question = question.lower()
        return any(word in question for word in keywords)

     

    def on_ai_response(self, answer):
        """This runs once the AI is done thinking."""
        # Remove the "Thinking..." message and add the real answer
        self.update_display("AI", answer)

    def update_display(self, sender, text):
        self.display.config(state='normal')
        # Simple logic to remove "Thinking..." when the real answer arrives
        content = self.display.get("1.0", tk.END)
        if "AI: Thinking..." in content and sender == "AI":
            # Delete the last "Thinking..." line
            self.display.delete("end-3l", "end-1l") 
        
        self.display.insert(tk.END, f"{sender}: {text}\n\n")
        self.display.config(state='disabled')
        self.display.yview(tk.END)

def main():
    dataset = load_dataset()
    print("--- IDS/IPS Scanner ---")
    ip = input("Enter IP address to scan: ").strip()
    if not ip: return

    flag, reason, action = ai_decision(ip, dataset)
    
    app = ChatGUI(ip, flag, reason, action)
    app.root.mainloop()

if __name__ == "__main__":
    main()

--- IDS/IPS Scanner ---


Enter IP address to scan:  1.1.1.1
